In [1]:
import cv2
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.core.protobuf import config_pb2
from tqdm import tqdm
import pickle

tf.reset_default_graph()

In [2]:
img_size = 240
batch_size = 62

In [3]:
# Placeholders for input data and the targets
X_noise = tf.placeholder(tf.float32, (None, img_size, img_size, 3), name='Input')
X_noise_bw = tf.placeholder(tf.float32, (None, img_size, img_size, 1), name='Input_bw')
X_target = tf.placeholder(tf.float32, (None, img_size, img_size, 3), name='Target')
# decoder_input = tf.placeholder(dtype=tf.float32, shape=[1, z_dim], name='Decoder_input')
test = tf.placeholder(tf.float32, (img_size,img_size,3), name='test')
test_bw = tf.placeholder(tf.float32, (img_size,img_size,1), name='test_bw')

In [4]:
def data(i):
    training_data=[]
    training_data_bw=[]
    for j in range((i*62),(62+(i*62))):
        image = cv2.imread('D:\satellitedata\shufflenoisydata\%d.jpg' %(j))
        new_array=cv2.resize(image,(img_size,img_size))
        norm_img = np.zeros((img_size,img_size))
        final_img = cv2.normalize(new_array,  norm_img, 0, 255, cv2.NORM_MINMAX)
        new_array = final_img
        new_array_bw = cv2.cvtColor(new_array, cv2.COLOR_BGR2GRAY)
        new_array_bw = cv2.resize(new_array_bw,(img_size,img_size))
        new_array_bw = np.reshape(new_array_bw, (img_size,img_size,1))
        training_data.append(new_array)
        training_data_bw.append(new_array_bw)
    return training_data, training_data_bw

In [5]:
def label(i):
    label = []
    for j in range((i*62),(62+(i*62))):
        image = cv2.imread('D:\\satellitedata\\shufflelabel\\%d.jpg' %(j))
        new_array=cv2.resize(image,(img_size,img_size))
        norm_img = np.zeros((img_size,img_size))
        final_img = cv2.normalize(new_array, norm_img, 0, 255, cv2.NORM_MINMAX)
        new_array = final_img
        label.append(new_array)
    return label

In [6]:
def generator(z, z_bw, reuse=None):
    with tf.variable_scope('gen',reuse=reuse):
        filters = {1:20,2:20,3:60, 4:60, 5:60, 6:60}
        conv1 = tf.layers.conv2d(z, filters[1], (3,3),strides=(2, 2), padding='valid',activation=None, use_bias=True, kernel_initializer='random_uniform', bias_initializer=tf.zeros_initializer())
        conv2 = tf.layers.conv2d(z_bw, filters[2], (3,3),strides=(2,2),padding='valid',activation=None,use_bias=True, kernel_initializer='random_uniform', bias_initializer=tf.zeros_initializer())
        conv = tf.concat([conv1, conv2], 3)
        conv3 = tf.layers.conv2d(conv, filters[3],(5,5), strides=(2, 2), padding='valid', activation=tf.nn.relu,use_bias=True, kernel_initializer='random_uniform', bias_initializer=tf.zeros_initializer())
        convD1 = tf.layers.conv2d(conv3,filters[4],(5,5), strides=(1, 1), padding='valid',activation=tf.nn.relu,dilation_rate=(2, 2), use_bias=True, kernel_initializer='random_uniform', bias_initializer=tf.zeros_initializer())
        upsample1 = tf.image.resize_nearest_neighbor(convD1, (196,196))
        conv4 = tf.layers.conv2d(upsample1, 40, (7,7), padding='same',activation=tf.nn.relu)
        upsample2 = tf.image.resize_nearest_neighbor(conv4, (img_size,img_size))
        conv5 = tf.layers.conv2d(upsample2, 3, (5,5), padding='same',activation=tf.nn.relu)
        return conv5
    
def discriminator(X,reuse=None):
    with tf.variable_scope('dis',reuse=reuse):
        conv1 = tf.layers.conv2d(X, 20, (3,3),strides=(2, 2), padding='valid',activation=tf.nn.relu, use_bias=True, kernel_initializer='random_uniform', bias_initializer=tf.zeros_initializer())
        conv2 = tf.layers.conv2d(conv1,40, (3,3),strides=(2,2),padding='valid',activation=tf.nn.relu,use_bias=True, kernel_initializer='random_uniform', bias_initializer=tf.zeros_initializer())
        flat = tf.contrib.layers.flatten(conv2)
        logits=tf.layers.dense(flat,units=1)
        output=tf.sigmoid(logits)
        
        return output,logits

In [7]:
# real_images=tf.placeholder(tf.float32,shape=[None,784])
# z=tf.placeholder(tf.float32,shape=[None,100])

G=generator(X_noise, X_noise_bw)
D_output_real,D_logits_real=discriminator(X_target)
D_output_fake,D_logits_fake=discriminator(G,reuse=True)

Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Please use `layer.__call__` method instead.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.Dense instead.


In [8]:
def loss_func(logits_in,labels_in):
    return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits_in,labels=labels_in))

D_real_loss=loss_func(D_logits_real,tf.ones_like(D_logits_real)*0.9) #Smoothing for generalization
D_fake_loss=loss_func(D_logits_fake,tf.zeros_like(D_logits_real))
D_loss=D_real_loss+D_fake_loss

G_loss= loss_func(D_logits_fake,tf.ones_like(D_logits_fake))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [9]:
lr=0.001

#Do this when multiple networks interact with each other
tvars=tf.trainable_variables()  #returns all variables created(the two variable scopes) and makes trainable true
d_vars=[var for var in tvars if 'dis' in var.name]
g_vars=[var for var in tvars if 'gen' in var.name]

D_trainer=tf.train.AdamOptimizer(lr).minimize(D_loss,var_list=d_vars)
G_trainer=tf.train.AdamOptimizer(lr).minimize(G_loss,var_list=g_vars)

batch_size=62
epochs=100
init=tf.global_variables_initializer()

In [10]:
samples=[] #generator examples
test = cv2.imread('test.png')
test = cv2.resize(test,(img_size,img_size))
norm_img = np.zeros((img_size,img_size))
test = cv2.normalize(test, norm_img, 0, 255, cv2.NORM_MINMAX)

test_bw = cv2.cvtColor(test, cv2.COLOR_BGR2GRAY )
test_bw = cv2.resize(test_bw,(img_size,img_size))
test_bw = np.reshape(test_bw, (img_size,img_size,1))
minibatch_Dloss = 0
minibatch_Gloss = 0
Gcost=[]
Dcost =[]
with tf.Session() as sess:
    sess.run(init)
    for epoch in tqdm(range(epochs)):
        num_batches=340
        for i in range(num_batches):
            X, X_bw = data(i)
            Y = label(i)
            _, temp_d_loss = sess.run([D_trainer,D_loss], feed_dict={X_target:Y,X_noise:X, X_noise_bw:X_bw})
            _, temp_g_loss = sess.run([G_trainer,G_loss], feed_dict={X_noise:X, X_noise_bw:X_bw})

            minibatch_Dloss += temp_d_loss / num_batches
            minibatch_Gloss += temp_g_loss / num_batches
        
            
        print("on epoch{}".format(epoch))
        # Print the cost every epoch
        print ("Cost after epoch %i: %f" % (epoch, minibatch_Dloss))
        print ("Cost after epoch %i: %f" % (epoch, minibatch_Gloss))
        Gcost.append(minibatch_Gloss)
        Dcost.append(minibatch_Dloss)
        
#         gen_sample=sess.run(generator(test, test_bw ,reuse=True),feed_dict={test:test, test_bw:test_bw})
#         samples.append(gen_sample)
        
# plt.imshow(samples[0].reshape(img_size,img_size))

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

on epoch0
Cost after epoch 0: 1.321992
Cost after epoch 0: 17.343468


  1%|▊                                                                            | 1/100 [27:51<45:58:15, 1671.67s/it]

on epoch1
Cost after epoch 1: 2.705177
Cost after epoch 1: 34.904203


  2%|█▌                                                                           | 2/100 [52:59<44:10:06, 1622.52s/it]

...

on epoch31
Cost after epoch 31: 138.573882
Cost after epoch 31: 84.284092


 32%|███████████████████████▎                                                 | 32/100 [13:17:58<28:15:18, 1495.86s/it]

on epoch32
Cost after epoch 32: 139.902416
Cost after epoch 32: 85.137110


 33%|████████████████████████                                                 | 33/100 [13:43:33<28:03:22, 1507.51s/it]

KeyboardInterrupt: 

In [ ]:
# plt.imshow(samples[99].reshape(img_size,img_size))

In [11]:
plt.plot(G_cost)

NameError: name 'G_cost' is not defined

In [ ]:
plt.plot(D_cost)